In [1]:
import pandas as pd
df = pd.read_csv('dataset/train.csv')  
df

/Applications/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


,class,image_path,name,xmax,xmin,ymax,ymin
0,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,797.0,701.0,262.0,211.0
1,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,932.0,786.0,329.0,238.0
2,3.0,4a48c42c9579ec0399e6c5a3e825e765.jpg,GARBAGE,736.0,657.0,275.0,229.0
3,7.0,ea906a663da6321bcef78be4b7d1afff.jpg,BAD_BILLBOARD,986.0,786.0,136.0,0.0
4,8.0,1c7d48005a12d1b19261b8e71df7cafe.jpg,SAND_ON_ROAD,667.0,549.0,228.0,179.0
...,...,...,...,...,...,...,...
19945,4.0,081e7bb3832ec5bb25276db161a96274.jpg,CONSTRUCTION_ROAD,1025.0,600.0,408.0,148.0
19946,2.0,1ff38a7af7f13b1201d17c6e1829373a.jpg,POTHOLES,657.0,418.0,364.0,282.0
19947,2.0,1ff38a7af7f13b1201d17c6e1829373a.jpg,POTHOLES,507.0,338.0,436.0,283.0
19948,7.0,ac97490f13140fc1bfe613ec69301b34.jpg,BAD_BILLBOARD,956.0,713.0,110.0,25.0


In [3]:
df["class"].value_counts()

3.0     8597
4.0     2730
2.0     2625
9.0     2253
7.0     1555
0.0     1124
8.0      748
10.0     127
1.0      107
5.0       83
6.0        1
Name: class, dtype: int64

In [4]:
df.loc[df['class'] == 6.0]

,class,image_path,name,xmax,xmin,ymax,ymin
19353,6.0,53d3797457a0d2e3afe146e2f797e77e.jpg,BAD_STREETLIGHT,985.0,359.0,161.0,-47.0


In [5]:
df["name"].value_counts()

GARBAGE              8597
CONSTRUCTION_ROAD    2730
POTHOLES             2625
CLUTTER_SIDEWALK     2253
BAD_BILLBOARD        1555
GRAFFITI             1124
SAND_ON_ROAD          748
UNKEPT_FACADE         127
FADED_SIGNAGE         107
BROKEN_SIGNAGE         83
BAD_STREETLIGHT         1
Name: name, dtype: int64

In [6]:
df["image_path"].value_counts()

291d3a6d38674da0e4b05135008bceb1.jpg    22
8b3335738dc291770ffb3613810128f4.jpg    19
fec90b848c9b6c4bcc1ec562597c55d8.jpg    19
0a2bc0dc2371794509f4b776aff0dd88.jpg    19
270c3a56f6a9abef21d61aec1eb53e88.jpg    19
                                        ..
1596246bbf778e2120656ca0eb5c2272.jpg     1
1726f638d4df5e9140ee7a691d155220.jpg     1
fcdf201c3cbe3543809d35eed38e2e40.jpg     1
320c8cc65946b84f11348cfb24496fd8.jpg     1
4688a6fd763942a56f0ae6510393a932.jpg     1
Name: image_path, Length: 7874, dtype: int64

In [7]:
import pybboxes as pbx

W, H = 1920, 1080  # WxH of the image
def convert_to_yolo_format(x_min, y_min, x_max, y_max):
    yolo_bbox = (x_min, y_min, x_max, y_max)
    r = pbx.convert_bbox(yolo_bbox, from_type="coco", to_type="yolo", image_size=(W, H))
    r = ' '.join(map(str,r))

    return r

In [8]:
for index, row in df.iterrows():
    with open(f'annotation/{row["image_path"]}.txt',"a") as file:
        file.write(str(row["class"])+" "+ convert_to_yolo_format(row["xmin"], row["ymin"], row["xmax"], row["ymax"]) + " \n")

In [9]:
with open('annotation/291d3a6d38674da0e4b05135008bceb1.jpg.txt') as f:
    contents = f.read()
    print(contents)

9.0 0.07708333333333334 0.30185185185185187 0.19166666666666668 0.2740740740740741 
9.0 0.2794270833333333 0.30416666666666664 0.32447916666666665 0.275 
9.0 0.48880208333333336 0.32546296296296295 0.48072916666666665 0.2953703703703704 
3.0 0.7041666666666667 0.21712962962962962 0.4979166666666667 0.1675925925925926 
3.0 0.72578125 0.25925925925925924 0.5057291666666667 0.2 
3.0 0.6317708333333333 0.2273148148148148 0.45729166666666665 0.18981481481481483 
3.0 0.5791666666666667 0.2064814814814815 0.4322916666666667 0.17962962962962964 
3.0 0.47291666666666665 0.24305555555555555 0.36354166666666665 0.1935185185185185 
3.0 0.4171875 0.19675925925925927 0.3 0.16203703703703703 
3.0 0.453125 0.18425925925925926 0.33645833333333336 0.1574074074074074 
3.0 0.3880208333333333 0.18101851851851852 0.2875 0.16203703703703703 
3.0 0.4046875 0.23425925925925925 0.32083333333333336 0.1962962962962963 
3.0 0.3625 0.23981481481481481 0.2760416666666667 0.18888888888888888 
3.0 0.3033854166666667 0

In [10]:
with open('annotation/53d3797457a0d2e3afe146e2f797e77e.jpg.txt') as f:
    contents = f.read()
    print(contents)

6.0 0.4434895833333333 0.031018518518518518 0.5130208333333334 0.14907407407407408 

